In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:

train_data = pd.read_csv('C:\\Users\\moro\Desktop\\train.csv')
train_data.head() 
# we didn't use test_data because it didn't have the y_test (survived) 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
train_data.drop(columns=['Cabin'], inplace=True)  #  حذفنا cabin لانو عم يحتوي على كثير من القيم المفقودة 

In [5]:
train_data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [6]:
scaler = StandardScaler()
train_data[['Age', 'Fare']] = scaler.fit_transform(train_data[['Age', 'Fare']])

In [7]:
num_duplicates = train_data.duplicated().sum()
print(f"Number of duplicated rows: {num_duplicates}")
# إزالة البيانات المكررة


Number of duplicated rows: 0


In [8]:


# تحويل البيانات الفئوية إلى أرقام
label_encoder = LabelEncoder()
train_data = pd.get_dummies(train_data, columns=['Embarked'], drop_first=True)
train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])

train_data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,-0.565736,1,0,A/5 21171,-0.502445,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.663861,1,0,PC 17599,0.786845,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,-0.258337,0,0,STON/O2. 3101282,-0.488854,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.433312,1,0,113803,0.420730,0,1
4,5,0,3,"Allen, Mr. William Henry",1,0.433312,0,0,373450,-0.486337,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,-0.181487,0,0,211536,-0.386671,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",0,-0.796286,0,0,112053,-0.044381,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,-0.104637,1,2,W./C. 6607,-0.176263,0,1
889,890,1,1,"Behr, Mr. Karl Howell",1,-0.258337,0,0,111369,-0.044381,0,0


In [9]:

# فصل البيانات إلى x  و  y 
X = train_data.drop(columns=['Survived', 'Name', 'Ticket', 'PassengerId'])
y = train_data['Survived']

class_distribution = y.value_counts()
print("\nDistribution of the target variable (Survived):")
print(class_distribution)

# test and train 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_train_pred_rf = rf_model.predict(X_train)
y_val_pred_rf = rf_model.predict(X_val)

# تجربة النموذج الثاني: SVC
svc_model = SVC(kernel='linear', random_state=42)
svc_model.fit(X_train, y_train)
y_train_pred_svc = svc_model.predict(X_train)
y_val_pred_svc = svc_model.predict(X_val)

# تقييم النماذج باستخدام دقة الاسترجاع
precision_rf = precision_score(y_val, y_val_pred_rf)
recall_rf = recall_score(y_val, y_val_pred_rf)

precision_svc = precision_score(y_val, y_val_pred_svc)
recall_svc = recall_score(y_val, y_val_pred_svc)

# حساب الدقة
rf_train_accuracy = accuracy_score(y_train, y_train_pred_rf)
rf_val_accuracy = accuracy_score(y_val, y_val_pred_rf)

svc_train_accuracy = accuracy_score(y_train, y_train_pred_svc)
svc_val_accuracy = accuracy_score(y_val, y_val_pred_svc)

# عرض النتائج
print(f'RandomForest - Precision: {precision_rf}, Recall: {recall_rf}')
print(f'SVC - Precision: {precision_svc}, Recall: {recall_svc}')
print(f"RandomForest - Train Accuracy: {rf_train_accuracy}, Validation Accuracy: {rf_val_accuracy}")
print(f"SVC - Train Accuracy: {svc_train_accuracy}, Validation Accuracy: {svc_val_accuracy}")

# persion must the fp be 0 # للبحث
# recall must the fn be 0 #   طبية او حوادث


Distribution of the target variable (Survived):
0    549
1    342
Name: Survived, dtype: int64
RandomForest - Precision: 0.7746478873239436, Recall: 0.7432432432432432
SVC - Precision: 0.7536231884057971, Recall: 0.7027027027027027
RandomForest - Train Accuracy: 0.9789325842696629, Validation Accuracy: 0.8044692737430168
SVC - Train Accuracy: 0.7879213483146067, Validation Accuracy: 0.7821229050279329


In [10]:
rf_model = RandomForestClassifier(n_estimators=10,
                                  random_state=42,
                                  max_depth=3,
                                  min_samples_split=2,
                                  min_samples_leaf=2)
rf_model.fit(X_train, y_train)
y_train_pred_rf = rf_model.predict(X_train)
y_val_pred_rf = rf_model.predict(X_val)
rf_train_accuracy = accuracy_score(y_train, y_train_pred_rf)
rf_val_accuracy = accuracy_score(y_val, y_val_pred_rf)
precision_rf = precision_score(y_val, y_val_pred_rf)
recall_rf = recall_score(y_val, y_val_pred_rf)
print(f"RandomForest - Train Accuracy: {rf_train_accuracy}, Validation Accuracy: {rf_val_accuracy}")
print(f'RandomForest - Precision: {precision_rf}, Recall: {recall_rf}')

RandomForest - Train Accuracy: 0.8328651685393258, Validation Accuracy: 0.8100558659217877
RandomForest - Precision: 0.8225806451612904, Recall: 0.6891891891891891


In [11]:
svc_model = SVC(kernel='rbf',gamma=0.1, random_state=42)
svc_model.fit(X_train, y_train)
y_train_pred_svc = svc_model.predict(X_train)
y_val_pred_svc = svc_model.predict(X_val)
svc_train_accuracy = accuracy_score(y_train, y_train_pred_svc)
svc_val_accuracy = accuracy_score(y_val, y_val_pred_svc)
print(f"SVC - Train Accuracy: {svc_train_accuracy}, Validation Accuracy: {svc_val_accuracy}")
print(f'SVC - Precision: {precision_svc}, Recall: {recall_svc}')


SVC - Train Accuracy: 0.8356741573033708, Validation Accuracy: 0.8156424581005587
SVC - Precision: 0.7536231884057971, Recall: 0.7027027027027027


In [12]:
svc_model = SVC(kernel='rbf',gamma=0.2, random_state=42)
svc_model.fit(X_train, y_train)
y_train_pred_svc = svc_model.predict(X_train)
y_val_pred_svc = svc_model.predict(X_val)
svc_train_accuracy = accuracy_score(y_train, y_train_pred_svc)
svc_val_accuracy = accuracy_score(y_val, y_val_pred_svc)
print(f"SVC - Train Accuracy: {svc_train_accuracy}, Validation Accuracy: {svc_val_accuracy}")
print(f'SVC - Precision: {precision_svc}, Recall: {recall_svc}')


SVC - Train Accuracy: 0.8412921348314607, Validation Accuracy: 0.8156424581005587
SVC - Precision: 0.7536231884057971, Recall: 0.7027027027027027


In [13]:
svc_model = SVC(kernel='poly',degree=2, random_state=42)
svc_model.fit(X_train, y_train)
y_train_pred_svc = svc_model.predict(X_train)
y_val_pred_svc = svc_model.predict(X_val)
svc_train_accuracy = accuracy_score(y_train, y_train_pred_svc)
svc_val_accuracy = accuracy_score(y_val, y_val_pred_svc)
print(f"SVC - Train Accuracy: {svc_train_accuracy}, Validation Accuracy: {svc_val_accuracy}")
print(f'SVC - Precision: {precision_svc}, Recall: {recall_svc}')


SVC - Train Accuracy: 0.8174157303370787, Validation Accuracy: 0.8044692737430168
SVC - Precision: 0.7536231884057971, Recall: 0.7027027027027027


In [14]:
svc_model = SVC(kernel='poly',degree=3, random_state=42)
svc_model.fit(X_train, y_train)
y_train_pred_svc = svc_model.predict(X_train)
y_val_pred_svc = svc_model.predict(X_val)
svc_train_accuracy = accuracy_score(y_train, y_train_pred_svc)
svc_val_accuracy = accuracy_score(y_val, y_val_pred_svc)
print(f"SVC - Train Accuracy: {svc_train_accuracy}, Validation Accuracy: {svc_val_accuracy}")
print(f'SVC - Precision: {precision_svc}, Recall: {recall_svc}')


SVC - Train Accuracy: 0.8202247191011236, Validation Accuracy: 0.8268156424581006
SVC - Precision: 0.7536231884057971, Recall: 0.7027027027027027


In [15]:
best_model = rf_model if recall_rf > recall_svc else svc_model
best_model_name = 'RandomForest' if best_model == rf_model else 'SVC'
print(f'The best model is {best_model_name}')


The best model is SVC


In [16]:
from sklearn.model_selection import cross_val_score, GridSearchCV

# cross val score
cross_val_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
print(f'Cross-Validation Scores: {cross_val_scores}')
print(f'Mean CV Accuracy: {cross_val_scores.mean()}')

# تعديل المعاملات الفائقة باستخدام GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}

grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)
print(f'Best Hyperparameters: {grid_search.best_params_}')
print(f'Best CV Accuracy: {grid_search.best_score_}')


Cross-Validation Scores: [0.81564246 0.7752809  0.79775281 0.78651685 0.82022472]
Mean CV Accuracy: 0.799083547799887
Best Hyperparameters: {'max_depth': None, 'n_estimators': 200}
Best CV Accuracy: 0.8350386039796623


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline.fit(X_train, y_train)
y_pred_pipeline = pipeline.predict(X_val)

# تقييم النموذج من خلال خط المعالجة
precision_pipeline = precision_score(y_val, y_pred_pipeline)
recall_pipeline = recall_score(y_val, y_pred_pipeline)

print(f'Pipeline - Precision: {precision_pipeline}, Recall: {recall_pipeline}')


Pipeline - Precision: 0.7857142857142857, Recall: 0.7432432432432432
